# SMS Creative Request 

# 1. Import Pacakge  

In [2]:
from os import listdir
from os.path import isfile, join
import datetime as dt
import filepath     
import infrastructure
import os 
import pandas as pd  
import numpy as np
import time 
import smartsheet

                      

In [3]:
# get all three report to see what we need request for SC,TF and Jump page  

from ipynb.fs.full.Content_Request_Report_SMS import *
from ipynb.fs.full.TF_Content_Request_Report_SMS import * 


/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/Content_Request_Report_SMS.ipynb:51: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  "source": [
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

  Hitpath ID Request ID High Priority Submission Date  Due Date   \
0      13498                    False      2024-07-17 2024-07-22   
1      13349                    False      2024-07-17 2024-07-22   
2      11600                    False      2024-07-17 2024-07-22   
3      12710                    False      2024-07-17 2024-07-22   
4      13519                    False      2024-07-17 2024-07-22   

  Request Sent to Copywriter (Kellie)  \
0                                       
1                                       
2                                       
3                                       
4                                       

  Unapproved Content Document Link (Copywriter)  \
0                                                 
1                                                 
2                                                 
3                                                 
4                                                 

  Unapproved Jump Page Content (Copyw

# 2.Get Data 

In [4]:
# Read file for SC Request 
save_path = filepath.output_folder + "Content Request Reports"
sc_files = [f for f in listdir(save_path) if isfile(join(save_path, f)) & ('Content_Request_Report_' in f)]
    #lexi_files = [f for f in listdir(lexi_download_path) if isfile(join(lexi_download_path, f)) 
        #              & ('Lexi 3.0' in f) & ~("Year" in f)]

sc_paths = ["{}/{}".format(save_path,sc_version) for sc_version in sc_files]
times = [os.path.getmtime(path) for path in sc_paths]
# times_formatted = [time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(time_float)) for time_float in times]
most_recent_sc_request = sc_files[times.index(max(times))]
sc_path = "{}/{}".format(save_path,most_recent_sc_request)
# READ EXCEL AND GET THE SPECIFIC SHEET 
# Reading multiple sheets from an Excel file
sc_request =  pd.read_excel(pd.ExcelFile(sc_path), sheet_name= 'Stats by Offer')


In [5]:
# Read file for TF Request 
save_path = filepath.output_folder + "Content Request Reports"
sc_files = [f for f in listdir(save_path) if isfile(join(save_path, f)) & ('TF Content Request Report' in f)]

sc_paths = ["{}/{}".format(save_path,sc_version) for sc_version in sc_files]
times = [os.path.getmtime(path) for path in sc_paths]
# times_formatted = [time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(time_float)) for time_float in times]
most_recent_tf_request = sc_files[times.index(max(times))]
tf_path = "{}/{}".format(save_path,most_recent_tf_request)
tf_request =  pd.read_excel(pd.ExcelFile(tf_path), sheet_name= 'TF Content Requests')


In [6]:
# get jump page report 
download_path = filepath.download_folder 
jp_files = [f for f in listdir(download_path) if isfile(join(download_path, f))& ('jump_page_feedback_reports' in f)]
jp_paths = ["{}/{}".format(download_path,jp_version) for jp_version in jp_files]
times = [os.path.getmtime(path) for path in jp_paths]
times_formatted = [time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(time_float)) for time_float in times]
most_recent_jp = jp_files[times.index(max(times))]
jp_path = "{}/{}".format(download_path,most_recent_jp)
jp_request =  pd.read_excel(pd.ExcelFile(jp_path), sheet_name= 'Need Request - Jump Page')

# 3. Combine all Request 

In [7]:
# get sc_request
sc_request_simple = sc_request[['Hitpath Offer ID','Priority Order','# of Pieces Requested Per Shortcode','Shortcode Abbreviations','# of Shortcodes','Total Requests per Hitpath']]
sc_request_simple.rename(columns = {'Priority Order':'SC Priority Order'}, inplace = True)
# get tf_request 
request_tf = ['MFA', 'N3G', "FRH",'DA','TRS']
tf_request['Toll Free Abbreviations'] = '\n'.join(request_tf)
tf_request['TF Priority Order'] = tf_request['CTR50'].rank(ascending=False)
tf_request.rename({"Pieces to Request":"# of Pieces Requested Per Toll Free"}, axis = 1, inplace = True)
tf_request['Total Requests per Toll Free'] = len(request_tf) * tf_request['# of Pieces Requested Per Toll Free']
tf_request_simple = tf_request[['Hitpath Offer ID','Toll Free Abbreviations','TF Priority Order','# of Pieces Requested Per Toll Free','Total Requests per Toll Free']]
# get jp_request 
jp_request['combine_index'] = jp_request['Text CTR'].rank(ascending = False) + jp_request['Jump Page Click Rate'].rank(ascending = True)
jp_request['JP Prioirty Level'] = jp_request['combine_index'].rank(ascending = True)
jp_request_simple = jp_request[['Hitpath Offer ID','JP Prioirty Level']]
jp_request_simple['# of Pieces Requested Per Jump Page'] = 1    

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5663/2954520889.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc_request_simple.rename(columns = {'Priority Order':'SC Priority Order'}, inplace = True)
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5663/2954520889.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jp_request_simple['# of Pieces Requested Per Jump Page'] = 1


In [8]:
combined_request = sc_request_simple.merge(tf_request_simple, on = 'Hitpath Offer ID', how = 'outer').merge(jp_request_simple, on = 'Hitpath Offer ID', how = 'outer')
combined_request['SC Priority Order'] = combined_request['SC Priority Order'].fillna(combined_request['SC Priority Order'].max()+1)
combined_request['TF Priority Order'] = combined_request['TF Priority Order'].fillna(combined_request['TF Priority Order'].max()+1)
combined_request['JP Prioirty Level'] = combined_request['JP Prioirty Level'].fillna(combined_request['JP Prioirty Level'].max()+1)
combined_request['score'] =combined_request['SC Priority Order']/combined_request['SC Priority Order'].max() * 0.4 + combined_request['TF Priority Order']/ combined_request['TF Priority Order'].max() * 0.3+ combined_request['JP Prioirty Level'] / combined_request['JP Prioirty Level'].max() * 0.3
combined_request['Priority'] = combined_request['score'].rank(ascending = True)
combined_request.drop(['score','JP Prioirty Level','TF Priority Order','SC Priority Order'], axis = 1, inplace = True)
combined_request.sort_values(by = 'Priority', inplace = True)
combined_request['Hitpath Offer ID'] = combined_request['Hitpath Offer ID'].astype(str).str.replace('.0','')
combined_request['# of Pieces Requested Per Jump Page'] = combined_request['# of Pieces Requested Per Jump Page'].fillna(0)
combined_request['# of Pieces Requested Per Toll Free'] = combined_request['# of Pieces Requested Per Toll Free'].fillna(0)  
combined_request['# of Pieces Requested Per Shortcode'] = combined_request['# of Pieces Requested Per Shortcode'].fillna(0)

# 4. Find final request based on capacity and submit it to Smartsheet 

In [9]:
submission_df = finalize_submission_all_df(combined_request)
final_submission_df,stats_by_offer = make_pieces_request_all_df(submission_df, request_limit=50)
#final_submission_df.drop(['Request Type (Request)','Compliance & Additional Notes (Request)'], inplace = True, axis = 1)

  Hitpath ID Request ID High Priority Submission Date  Due Date   \
0       8838                    False      2024-07-17 2024-07-22   
1      13059                    False      2024-07-17 2024-07-22   
2      13504                    False      2024-07-17 2024-07-22   
3      11890                    False      2024-07-17 2024-07-22   
4      12611                    False      2024-07-17 2024-07-22   
5      13349                    False      2024-07-17 2024-07-22   
6      12110                    False      2024-07-17 2024-07-22   
7      11646                    False      2024-07-17 2024-07-22   
8      13089                    False      2024-07-17 2024-07-22   

  Request Sent to Copywriter (Kellie)  \
0                                       
1                                       
2                                       
3                                       
4                                       
5                                       
6                               

In [10]:
#sheet_id = 8253683024220036 # real sheet id 
sheet_id = 5426394364333956 # test sheet id 
#key = '7fQH9Go8sf6CruwY3g4HHzKHe6wFMzNbgyAYy' # back up key
key = 'wf3Jr9OaAT5OE8gE8yctPqfmzrv6dUFATUODD'
result = place_in_smartsheet(final_submission_df, key, sheet_id)

Adding Data to Smartsheet...
RX Rep Marley Smith m.smith@rxmg.com
RX Rep Mary Beth Howe m.howe@rxmg.com
RX Rep Rhiannon Selander r.selander@rxmg.com
RX Rep Sarah Bowe s.bowe@rxmg.com
RX Rep Jennifer Sporleder j.sporleder@rxmg.com
RX Rep Rhiannon Selander r.selander@rxmg.com
RX Rep Surya Hemanth s.hemanth@rxmg.com
RX Rep Hartley Goode h.goode@rxmg.com
RX Rep Hartley Goode h.goode@rxmg.com
Data added to Smartsheet


In [11]:
combined_request

,Hitpath Offer ID,# of Pieces Requested Per Shortcode,Shortcode Abbreviations,# of Shortcodes,Total Requests per Hitpath,Toll Free Abbreviations,# of Pieces Requested Per Toll Free,Total Requests per Toll Free,# of Pieces Requested Per Jump Page,Priority
0,13059,2.0,HZB\nMBC\nSVT\nFLC\nDSS\nUAA,6.0,12.0,NaN,0.0,NaN,1.0,1.0
1,13349,2.0,HZB\nMBC\nSVT\nFLC\nDSS\nUAA,6.0,12.0,MFA\nN3G\nFRH\nDA\nTRS,2.0,10.0,0.0,2.0
4,12610,2.0,HZB\nMBC\nSVT\nFLC\nDSS\nUAA,6.0,12.0,NaN,0.0,NaN,1.0,3.0
6,13089,2.0,MBC\nFLC\nDSS\nHZB,4.0,8.0,NaN,0.0,NaN,1.0,4.0
16,12145,0.0,NaN,NaN,NaN,MFA\nN3G\nFRH\nDA\nTRS,2.0,10.0,1.0,5.0
15,13387,0.0,NaN,NaN,NaN,MFA\nN3G\nFRH\nDA\nTRS,2.0,10.0,1.0,6.0
2,11646,2.0,HZB\nMBC\nSVT\nFLC\nDSS\nUAA,6.0,12.0,NaN,0.0,NaN,0.0,7.0
9,12152,0.0,NaN,NaN,NaN,MFA\nN3G\nFRH\nDA\nTRS,2.0,10.0,0.0,8.0
18,12587,0.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,9.0
19,10086,0.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,10.0


# 5. Send email to the team

In [12]:
# Update content request report 

with pd.ExcelWriter(sc_path, mode='a', if_sheet_exists='replace',engine='openpyxl') as writer:
    stats_by_offer.to_excel(writer, sheet_name='Stats by Offer', index=False)
with pd.ExcelWriter(sc_path, mode='a', if_sheet_exists='replace',engine='openpyxl') as writer:
    final_submission_df.to_excel(writer, sheet_name='Content Requests', index=False)

In [13]:
from nltk import flatten
from openpyxl.styles import numbers
from openpyxl import Workbook
from openpyxl.styles import Font, Color, colors, fills
from openpyxl.utils.dataframe import dataframe_to_rows

from openpyxl.styles.borders import Border, Side, BORDER_THIN
thin_border = Border(
    left=Side(border_style=BORDER_THIN, color='D3D3D3'),
    right=Side(border_style=BORDER_THIN, color='D3D3D3'),
    top=Side(border_style=BORDER_THIN, color='D3D3D3'),
    bottom=Side(border_style=BORDER_THIN, color='D3D3D3')
)

from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment

In [14]:
# Make Recently Tested Content Report

submission_df_hitpaths = final_submission_df['Hitpath Offer ID'].tolist()
submission_df_hitpaths = [str(x) for x in submission_df_hitpaths]
stats_by_offer_hitpaths = stats_by_offer['Hitpath Offer ID'].tolist()
stats_by_offer_hitpaths = [str(x) for x in stats_by_offer_hitpaths]
hitpath_requests_removed = [hitpath for hitpath in stats_by_offer_hitpaths if hitpath not in submission_df_hitpaths]

# Load the workbook and the sheet
workbook = load_workbook(sc_path)
stats_by_offer_sheet = workbook['Stats by Offer']
content_request_sheet = workbook['Content Requests']
ws_headers = {}
ws_headers['Stats by Offer'] = stats_by_offer.columns.tolist()
ws_headers['Content Requests'] = final_submission_df.columns.tolist()
currency_format_cols = ['Revenue', 'eCPM', 'eCPM_min_target']
percent_format_cols = ['OOR', 'CTR', 'CTR_min_target', '30:90 Ratio']
comma_format_cols = ['Optout', 'Clicks', 'Delivered']

# Make entire row orange if offer was removed from requests
for row in stats_by_offer_sheet.iter_rows(min_row=2):
    # If the Sendable column if False, make entire row orange
    if row[ws_headers['Stats by Offer'].index('Hitpath Offer ID')].value in hitpath_requests_removed:
        for cell in row:     
            cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='fce5cd'))
            cell.border = thin_border
for ws in [stats_by_offer_sheet,content_request_sheet]:
    dims = {}
    headers = ws_headers[ws.title]

    for row in ws.rows:
        for cell in ws["1:1"]:
            cell.font = Font(bold=True)
        for cell in row:
            newline_count = 1
            if cell.value:
                if type(cell.value)==str:
                    if ('\n' in cell.value):
                        newline_count = cell.value.count('\n')
                dims[cell.column_letter] = max((dims.get(cell.column_letter, 0), len(str(cell.value))))/newline_count 
    for col, value in dims.items():
        ws.column_dimensions[col].width = value
    for col in currency_format_cols:
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = numbers.FORMAT_CURRENCY_USD_SIMPLE

    for col in percent_format_cols:    
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = numbers.FORMAT_PERCENTAGE_00

    for col in comma_format_cols:    
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = '#,##0'
workbook.save(sc_path)

In [15]:
# comment by Lili on 7/12
#email

toaddr = ['offernotices@rxmg.com', 'nathan@rxmg.com','nina@rxmg.com', 'k.smith@rxmg.com', 'a.miller@rxmg.com',  'lili@rxmg.com', 'r.woodward@rxmg.com', 'b.ratzlaff@rxmg.com']
#toaddr = ['lili@rxmg.com']
subject_line = f"SMS Content Request Report - {today}"
email_body = "Hi Team,\n\n"
email_body += "Please find this week's Content Request Report attached.\n\n" 
email_body += "The following AMs have offer content that was requested:\n" 
for key, value in rx_rep_offer_dict.items():
    if value == 1:
        email_body += f"\t{key}: {value} offer\n"
    else:
        email_body += f"\t{key}: {value} offers\n"
email_body += f"\nThanks,\n{filepath.name}\n\n"
print(email_body)
#send email
for i in toaddr:
    send_email.send_email([sc_path], subject_line, email_body, i)


Hi Team,

Please find this week's Content Request Report attached.

The following AMs have offer content that was requested:
	Hartley Goode: 1 offer
	Mary Beth Howe: 1 offer

Thanks,
Lili Guo




connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP af79cd13be357-7a160c636ecsm438193485a.94 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP af79cd13be357-7a160c636ecsm438193485a.94 - gsmtp'
connect: b'smtp.gmail.com ESMTP af79cd13be357-7a160c636ecsm438193485a.94 - gsmtp'
send: 'ehlo 1.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.ip6.arpa\r\n'
reply: b'250-smtp.gmail.com at your service, [47.145.216.112]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [47.145.216.112]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo 1.0.0.0.0.0.0.0.0.0.0.0.0.0